## Dyalog recipes

To see a correct render of this notebook: https://nbviewer.jupyter.org/github/xpqz/DyalogCookbook/blob/master/Dyalog%20Cookbook.ipynb

This notebook contains my notes from learning APL. A lot of these has come from helpful discussions on the APL Orchard Stack Exchange chatroom.

APL allows you to configure the index origin, which is to say if arrays and vectors start at index 0 or 1. The default is 1, and in this notebook this is what we use, to avoid confusion. If you're used to 0-indexed languages (C, Java, Python etc) this may grate, but in APL it soon feels natural.

In [14]:
⎕IO←1 ⍝ one-based indexing (this is the default, but let's be explicit)
]box on -style=max -trains=tree -fns=on ⍝ Pass all output through DISPLAY

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

### Regular expressions

Dyalog supports the full PCRE syntax via ⎕S and ⎕R

See http://archive.vector.org.uk/art10500870

In [3]:
⍝ Get all words. The config-string '&' returns the matched string.
'\w+' ⎕S '&' ⊣ 'the cat sat on the mat'

┌→───────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ ┌→──┐ ┌→──┐ │
│ │the│ │cat│ │sat│ │on│ │the│ │mat│ │
│ └───┘ └───┘ └───┘ └──┘ └───┘ └───┘ │
└∊───────────────────────────────────┘

In [4]:
⍝ Reverse each word
'\w+' ⎕S {⌽⍵.Match} ⊣ 'the cat sat on the mat'

┌→───────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ ┌→──┐ ┌→──┐ │
│ │eht│ │tac│ │tas│ │no│ │eht│ │tam│ │
│ └───┘ └───┘ └───┘ └──┘ └───┘ └───┘ │
└∊───────────────────────────────────┘

Note that ⎕S is a dyadic function that returns a function. Left arg is always the pattern. Right arg is either a transformation function or a config string.

In [5]:
⍝ Capture groups
'c([^t]+)t' ⎕S '\1' ⊣ 'The cabriolet parked on the road'

┌→──────────┐
│ ┌→──────┐ │
│ │abriole│ │
│ └───────┘ │
└∊──────────┘

The config string is used to specify the capture group following Perl conventions, \1, \2 etc

The ⎕R function does a replace:

In [7]:
'[si]'⎕R'S' ⊢ 'mississippi'

┌→──────────┐
│mSSSSSSSppS│
└───────────┘

In [8]:
's'⎕R'S'⍠'ML' 2 ⊢ 'mississippi' ⍝ Set the "match limit" to 2, with the variant operator ⍠

┌→──────────┐
│miSSissippi│
└───────────┘

In [9]:
{⊃('.+' ⎕S '\u0')⍵} 'Hello All People in The World' ⍝ Upper-case a string

┌→────────────────────────────┐
│HELLO ALL PEOPLE IN THE WORLD│
└─────────────────────────────┘

Here's a dfn that takes a regex and a string and splits the latter on the former:

In [10]:
RegSplit←{⊃{⍵/⍨⍺∨¯1⌽⍺}/↓⍉↑('(.*?)',⍺)'(.*?)$'⎕S{⍵.((~PatternNum)(Lengths[2]↑Offsets[2]↓Block))}⊢⍵}
RegSplit2←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵} ⍝ faster

In [11]:
'/+' RegSplit 'Here///be/dragons/'  ⍝ Note empty last item in result
'/+' RegSplit2 'Here///be/dragons/'  ⍝ Note empty last item in result

┌→──────────────────────────┐
│ ┌→───┐ ┌→─┐ ┌→──────┐ ┌⊖┐ │
│ │Here│ │be│ │dragons│ │ │ │
│ └────┘ └──┘ └───────┘ └─┘ │
└∊──────────────────────────┘

┌→──────────────────────────┐
│ ┌→───┐ ┌→─┐ ┌→──────┐ ┌⊖┐ │
│ │Here│ │be│ │dragons│ │ │ │
│ └────┘ └──┘ └───────┘ └─┘ │
└∊──────────────────────────┘

## FizzBuzz

FizzBuzz is a well-known task sometimes set at interviews for coding jobs. The idea is to consider a sequence o f numbers and print "FizzBuzz" if the number is divisible by 3 and 5, "Buzz" if divisible by "5", "Fizz" if divisible by 3 and otherwise just output the number.

Here's APL-FizzBuzz, as given in https://rosettacode.org/wiki/FizzBuzz#APL

In [40]:
{⊃⍵ 'Fizz' 'Buzz' 'FizzBuzz'[⎕IO++/1 2×0=3 5|⍵]}¨⍳16

┌→───────────────────────────────────────────────────────────────────────────┐
│ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───────┐ │
│ 1 2 │Fizz│ 4 │Buzz│ │Fizz│ 7 8 │Fizz│ │Buzz│ 11 │Fizz│ 13 14 │FizzBuzz│ 16 │
│ └────┘ └────┘ └────┘ └────┘ └────┘ └────┘ └────────┘ │
└∊───────────────────────────────────────────────────────────────────────────┘

The indexing function `1++/1 2×0=3 5|⍵` generates 1 for numbers not divisible by either 3 or 5, 2 for divisible by 3, 3 for divisible by 5 and 4 for divisible by 3 and 5. So how does that work?

Consider it right to left:

In [16]:
{3 5|⍵}¨10 11 15 ⍝ Return length 2 vector with remainder from div by 3 and 5

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 0│ │2 1│ │0 0│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

Next step is to pick out the zeros

In [17]:
{0= 3 5|⍵}¨10 11 15 ⍝ Return length 2 vector with remainder from div by 3 and 5, highlighing zeros

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 1│ │0 0│ │1 1│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

...and multiply by 1 and 2 respectively:

In [18]:
{1 2× 0= 3 5| ⍵}¨10 11 15 ⍝ ...multiplied by 1 and 2

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 2│ │0 0│ │1 2│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

Now what remains is to sum that up, and add `⎕IO`:

In [41]:
{⎕IO++/1 2× 0= 3 5|⍵}¨10 11 15 ⍝ ...multiplied by 1 and 2, and summed

┌→────┐
│3 1 4│
└~────┘

We can now index into the vector containing our output text, after disclosing:

In [42]:
{⊃⍵ 'Fizz' 'Buzz' 'FizzBuzz'[⎕IO++/ 1 2× 0= 3 5| ⍵]} ¨ 3 10 11 15

┌→────────────────────────────┐
│ ┌→───┐ ┌→───┐ ┌→───────┐ │
│ │Fizz│ │Buzz│ 11 │FizzBuzz│ │
│ └────┘ └────┘ └────────┘ │
└∊────────────────────────────┘

This version is iterative, as it uses each (`¨`). We can take a radically different approach to take advantage of APL's array operators:

In [5]:
]dinput
FizzBuzz←{
    nums←⍳⍵
    mat←(⍱⌿⍪⊢)0=3 5∘.|nums  ⍝ Remainder matrix
    mat×@1⍨←nums            ⍝ Multiply first row by the source numbers
    mat←(⊂'Fizz')@⊢@2⊢mat   ⍝ Replace any 1s in second row by Fizz
    mat←(⊂'Buzz')@⊢@3⊢mat   ⍝ Replace any 1s in third row by Buzz
    0~⍨¨,⌿mat               ⍝ Merge downwards and remove any zeros
}

In [6]:
FizzBuzz 16

┌→─────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→┐ ┌→┐ ┌→───┐ ┌→┐ ┌→───┐ ┌→───┐ ┌→┐ ┌→┐ ┌→───┐ ┌→───┐ ┌→─┐ ┌→───┐ ┌→─┐ ┌→─┐ ┌→───────┐ ┌→─┐ │
│ │1│ │2│ │Fizz│ │4│ │Buzz│ │Fizz│ │7│ │8│ │Fizz│ │Buzz│ │11│ │Fizz│ │13│ │14│ │FizzBuzz│ │16│ │
│ └~┘ └~┘ └────┘ └~┘ └────┘ └────┘ └~┘ └~┘ └────┘ └────┘ └~─┘ └────┘ └~─┘ └~─┘ └────────┘ └~─┘ │
└∊─────────────────────────────────────────────────────────────────────────────────────────────┘

### Fibonacci sequence

A classic number sequence: https://en.wikipedia.org/wiki/Fibonacci_number

A direct, tail-recursive implementation

In [31]:
]dinput
Fib1←{
    ⍺←0 1
    ⍵=0:⍺
    (⍺,+/¯2↑⍺)∇⍵-1
}

In [32]:
Fib1 10

┌→───────────────────────────┐
│0 1 1 2 3 5 8 13 21 34 55 89│
└~───────────────────────────┘

Following http://www.haskellforall.com/2020/04/blazing-fast-fibonacci-numbers-using.html we can implement 
a fast fibonacci generator using the matrix form of the sequence, and via exponentiation by squaring.

In [33]:
]dinput
Fib2←{      ⍝ https://en.wikipedia.org/wiki/Fibonacci_number#Matrix_form  
    0=⍵:0
    Ebs←{  ⍝ https://en.wikipedia.org/wiki/Exponentiation_by_squaring
        ((,⍨⍴1,⍴∘0)≢⍺) {             ⍝ Identity matrix of the same rank as ⍺
            (x n)←⍵
            n<0:⍺ ⋄ n=1:x+.×⍺
            0=2|n:⍺∇(x+.×x) (n÷2)
            x+.×⍺∇(x+.×x) ((n-1)÷2)
        } ⍺ (⍵)
    }
    m←2 2⍴0 1 1 1
    1 0 +.× (m Ebs ⍵) +.× 0 1
}

In [34]:
Fib2¨0,⍳20

┌→───────────────────────────────────────────────────────────────────┐
│0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765│
└~───────────────────────────────────────────────────────────────────┘

### Split a string into a vector of a single char and a number

E.g 'X1234' gives 'X' 1234 -- via @Adám on APLOrchard

In [35]:
1(↑,∘⍎↓)'X1234' ⍝ Drop the first char, evaluate the rest, and catenate the first char.

┌→─────┐
│X 1234│
└+─────┘

Of course, using ⍎ on unfiltered user input can be dangerous. 

In [36]:
(⊃,∘⍎∩∘⎕D)'X4234' ⍝ safer, but can throw an error.

┌→─────┐
│X 4234│
└+─────┘

Here the [⎕D](http://help.dyalog.com/latest/#Language/System%20Functions/d.htm) is a system variable specifying "digits":

In [37]:
⎕D

┌→─────────┐
│0123456789│
└──────────┘

The industrial strength version uses [⎕VFI](http://help.dyalog.com/latest/#Language/System%20Functions/vfi.htm), (Verify and Fix Input) to properly -- and safely -- parse the integer part.

In [45]:
1(↑,2⊃∘⎕VFI↓)'X1234' 

┌→─────┐
│X 1234│
└+─────┘

Verify and Fix Input, when used dyadically, can be used to split and convert strings:

In [47]:
2⊃'x'⎕VFI'19x29x21'

┌→───────┐
│19 29 21│
└~───────┘

We can see how the first version is derived from a perhaps more obvious starting point:

In [48]:
{a←1↑⍵ ⋄ b←⍎1↓⍵ ⋄ a,b} 'X1234' ⍝ Take head, eval tail

{(1↑⍵),(⍎1↓⍵)} 'X1234'         ⍝ substitute variables
1 {(⍺↑⍵),(⍎⍺↓⍵)} 'X1234'       ⍝ break out 1
1 ((⊣↑⊢),(⍎⊣↓⊢)) 'X1234'       ⍝ train
1 ((↑),(⍎↓)) 'X1234'           ⍝ simplify
1 (↑,∘⍎↓) 'X1234'              ⍝ remove parens

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

┌→─────┐
│X 1234│
└+─────┘

### Partition a vector into groups of equal elements

In [51]:
partition←{⍵⊂⍨1,2≠/⍵}
partition 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

┌→────────────────────────────────────────────┐
│ ┌→────────┐ ┌→──┐ ┌→┐ ┌→──────┐ ┌→──┐ ┌→──┐ │
│ │1 1 1 1 1│ │2 2│ │1│ │4 4 4 4│ │1 1│ │2 2│ │
│ └~────────┘ └~──┘ └~┘ └~──────┘ └~──┘ └~──┘ │
└∊────────────────────────────────────────────┘

If we need the run lengths, we can just count them up:

In [52]:
≢¨partition 1 1 1 1 1 2 2 1 4 4 4 4 1 1 2 2

┌→──────────┐
│5 2 1 4 2 2│
└~──────────┘

### Match brackets

In [58]:
{+\1 ¯1 0['()'⍳⍵]} '((2×3)+4)' 

┌→────────────────┐
│1 2 2 2 2 1 1 1 0│
└~────────────────┘

Nice! That snippet was supposedly penned by Ken himself. 

https://forums.dyalog.com/viewtopic.php?f=30&t=1616
https://www.jsoftware.com/papers/perlis78.htm

How does it work?

Well, the expression `'()'⍳⍵` creates a vector of the same length as the argument containing a 1 (or 0, depending on `⎕io`) if the corresponding character is a '(', 2 for ')' and 3 (overflow) for everything else. We then map those values to 1, ¯1 and 0 respectively, and scan-sum over those.

Another way, slightly shorter, to achieve the same thing:

In [59]:
{+\-⌿'()'∘.=⍵} '((2×3)+4)'  ⍝ Match brackets

┌→────────────────┐
│1 2 2 2 2 1 1 1 0│
└~────────────────┘

This version builds a matrix where the two rows are binary masks showing the positions of the respective brackets.

In [60]:
'()'∘.='((2×3)+4)'

┌→────────────────┐
↓1 1 0 0 0 0 0 0 0│
│0 0 0 0 0 1 0 0 1│
└~────────────────┘

Next step is to subtract row 2 from row 1

In [61]:
-⌿'()'∘.='((2×3)+4)'

┌→──────────────────┐
│1 1 0 0 0 ¯1 0 0 ¯1│
└~──────────────────┘

and then scan-sum

In [62]:
+\-⌿'()'∘.='((2×3)+4)'

┌→────────────────┐
│1 2 2 2 2 1 1 1 0│
└~────────────────┘

## Stdlib

In [12]:
Zip←{(⊂⍋∊⍳∘≢¨⍺ ⍵)⌷⍺⍪⍵}                             ⍝ Zip two arrays

In [ ]:
Unzip←|∘⍳∘≢⊢∘⊂⌸⊢                                   ⍝ 2 Unzip 'dyalog'

In [ ]:
Split←≠⊆⊢                                          ⍝ Split string on a char

In [13]:
RegSplit←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵} ⍝ Split string on a pattern

In [14]:
Range←⊣+∘⍳-⍨                                       ⍝ start Range end

In [ ]:
Foldl←{↑⍺⍺⍨/(⌽⍵),⊂⍺}                               ⍝ 0 + Foldl 1 2 3 4 5 ⍝ Must give initial accumulator state

In [ ]:
Pairs←{,⍳⍵ ⍵}

In [15]:
Balance←{+\-⌿⍺∘.=⍵}                                ⍝ '()' Balance '(a+b+(1-c))'

In [7]:
Md5←{⎕SH 'md5 -q -s "',⍵,'"'}

In [ ]:
Normalise←⊢÷+/                                     ⍝ Make vector components sum to 1

### Error handling

Using error guards

In [63]:
4 {3::'out of bounds' ⋄ ⍺ ⌷ ⍵} 1 2 3 4 5 6 7

4

In [64]:
8 {3::'out of bounds' ⋄ ⍺ ⌷ ⍵} 1 2 3 4 5 6 7

┌→────────────┐
│out of bounds│
└─────────────┘

### Sets

Set XOR - union minus intersection, using a train

In [66]:
⊃(∪~∩)/ (1 2 3 4 5 6) (4 5 6 7 8) ⍝ Elements in ⍺ or ⍵ but not in both

┌→────────┐
│1 2 3 7 8│
└~────────┘

### Encode and decode ⊤ ⊥

Encode and decode converts between decimal and some encoding vector.

Split a number into digits and pad with zeros from the left:

In [67]:
10 10 10 10 10 ⊤ 12

┌→────────┐
│0 0 0 1 2│
└~────────┘

Convert from seconds to H M S:

In [68]:
0 24 60 60 ⊤ 8473

┌→────────┐
│0 2 21 13│
└~────────┘

Decode goes the other way

In [69]:
0 24 60 60 ⊥ 0 2 21 13 

8473

Sometimes swapped to save some brackets

In [70]:
12 ⊤⍨ 5⍴10 ⍝ instead of (5⍴10) ⊤ 12

┌→────────┐
│0 0 0 1 2│
└~────────┘

We can exploit decode to sum elements of a vector, which faster than the normal +/ approach and a useful trick in trains.

In [71]:
1⊥1 3 2 5 6

17

## Trains

Trains are APL's functional composition mechanism.

In [72]:
⍝ Construct a vector of the sum and product of two numbers
2 (+,×) 5

┌→───┐
│7 10│
└~───┘

This is a three-function train, consisting of sum, catenate, product. For a three-function train (LMR) in the dyadic case, the execution flow is (⍺ L ⍵) M (⍺ R ⍵) or specifically (2+5),(2×5).

Here's another example: split a string on a separator:

In [73]:
',' (≠⊆⊢) 'one,two,three'

┌→────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→────┐ │
│ │one│ │two│ │three│ │
│ └───┘ └───┘ └─────┘ │
└∊────────────────────┘

Given the execution flow above we have:

In [74]:
Lhs ← ','≠'one,two,three'  ⍝ Binary match vector
Rhs ← ','⊢'one,two,three'  ⍝ Just return the right-hand argument
Lhs ⊆ Rhs                  ⍝ Partition based on binary match vector

┌→────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→────┐ │
│ │one│ │two│ │three│ │
│ └───┘ └───┘ └─────┘ │
└∊────────────────────┘

The right-tack ⊢ is used in trains as an identity function to force the correct monadic/dyadic behaviour.

Another canonical example is calculating the mean with a three-function monadic train:

In [75]:
(+/÷≢) 3 6 4 9

5.5

This translates to:

In [76]:
Lhs ← +/ 3 6 4 9   ⍝ Sum 
Rhs ← ≢ 3 6 4 9    ⍝ Count
Lhs ÷ Rhs          ⍝ Mean

5.5

Set XOR using a union-not-intersect train

In [77]:
⊃(∪~∩)/ (1 2 3 4 5 6) (4 5 6 7 8)

┌→────────┐
│1 2 3 7 8│
└~────────┘

Here's a longer example: generate an integer range

In [79]:
¯1+5 (⊣+∘⍳-⍨) 10

┌→────────┐
│5 6 7 8 9│
└~────────┘

and an even longer, splitting a vector in n parts by unzipping:

In [80]:
2 (|∘⍳∘≢⊢∘⊂⌸⊢) 'dyaloge'

┌→─────────────┐
│ ┌→───┐ ┌→──┐ │
│ │daoe│ │ylg│ │
│ └────┘ └───┘ │
└∊─────────────┘

## Tack tricks

Left (⊣) and right (⊢) tacks return the argument they point to.

In [81]:
'Left'⊣'Right' ⋄ 'Left'⊢'Right'

┌→───┐
│Left│
└────┘
┌→────┐
│Right│
└─────┘

We saw earlier their place in trains to refer to left or right arguments.

In [82]:
',' (≠⊆⊢) 'one,two,three'

┌→────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→────┐ │
│ │one│ │two│ │three│ │
│ └───┘ └───┘ └─────┘ │
└∊────────────────────┘

Another use is when we want to return a value after first mutating it.

In [83]:
mem ← 1 2 3 4 5
mem ⊣ mem[1 2] ← 10 10

┌→──────────┐
│10 10 3 4 5│
└~──────────┘

We can also use tacks to pick the first or last column of a matrix. This is a Dyalog [idiom](https://help.dyalog.com/17.1/#Language/Defined%20Functions%20and%20Operators/Idiom%20Recognition/Idiom%20List.htm)
meaning it's highly optimised.

In [84]:
⎕←m←2 3⍴1 2 3 4 5 6 7 8 9
⎕←first←⊣/m
⎕←last←⊢/m

┌→────┐
↓1 2 3│
│4 5 6│
└~────┘

┌→──┐
│1 4│
└~──┘

┌→──┐
│3 6│
└~──┘

## Indexing

There are several ways of indexing into arrays and vectors. Crucially, elements of vectors and matrices are always scalars, but a scalar can be a boxed-up vector or matrix.

Indexing with [] or ⌷ returns the box, not the element, although if the element is a simple scalar, it's the same thing.

In [85]:
v←⍳10
v[5]
v[5 2]
3⌷v

5

┌→──┐
│5 2│
└~──┘

3

In [86]:
v←(1 2 3)(4 5 6)(7 8 9)
v[1]  ⍝ Return the box at element 1

┌─────────┐
│ ┌→────┐ │
│ │1 2 3│ │
│ └~────┘ │
└∊────────┘

To get to the boxed element, we need to either disclose, or pick:

In [87]:
⊃v[1]  ⍝ Open box (disclose)
1⊃v    ⍝ Pick element at 1

┌→────┐
│1 2 3│
└~────┘

┌→────┐
│1 2 3│
└~────┘

The same box-unbox rules also apply to mutation:

In [90]:
v[2]←1 2 3 ⍝ Will fail with LENGTH ERROR, as value isn't boxed.

LENGTH ERROR
      v[2]←1 2 3 ⍝ Will fail with LENGTH ERROR, as value isn't boxed.
          ∧


Instead we need to explicitly enclose the new vector:

In [92]:
v[2]←⊂1 2 3
v

┌→────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→────┐ │
│ │1 2 3│ │1 2 3│ │7 8 9│ │
│ └~────┘ └~────┘ └~────┘ │
└∊────────────────────────┘

Set multiple values with @ without mutation

In [97]:
9 9 @ 3 4 ⊢ vec←1 2 3 4 5 6 7 8             ⍝ Note -- no mutation
vec 

┌→──────────────┐
│1 2 9 9 5 6 7 8│
└~──────────────┘

┌→──────────────┐
│1 2 3 4 5 6 7 8│
└~──────────────┘

In [98]:
vec ⊣ vec[4 5] ← 8 8          ⍝ Note -- direct assignment mutates v

┌→──────────────┐
│1 2 3 8 8 6 7 8│
└~──────────────┘

The @ operator can also apply functions (via @Adám on APLOrchard):

In [99]:
(-@2 5)10 20 30 40 50 60  ⍝ Apply monadic - at positions 2 and 5

┌→──────────────────┐
│10 ¯20 30 40 ¯50 60│
└~──────────────────┘

which also works for dyads:

In [100]:
7(+@2 5)10 20 30 40 50 60 

┌→────────────────┐
│10 27 30 40 57 60│
└~────────────────┘

In [101]:
'x'@(∊∘⎕A)'Hello World' ⍝ Replace all uppercase letters with 'x'. ⎕A gives the uppercase letters.

┌→──────────┐
│xello xorld│
└───────────┘

## Products

Cartesian product

In [105]:
pairs ← {,(⍳⍵)∘.,⍳⍵}  ⍝ Python [[x, y] for x in range(1,n+1) for y in range(1,n+1)]
pairs 3

┌→──────────────────────────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 1│ │1 2│ │1 3│ │2 1│ │2 2│ │2 3│ │3 1│ │3 2│ │3 3│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊──────────────────────────────────────────────────────┘

or an alternative formulation, using iota. Think of the arguments to iota as defining the shape of the result:

In [106]:
pairs ← {,⍳⍵ ⍵}
pairs 3

┌→──────────────────────────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 1│ │1 2│ │1 3│ │2 1│ │2 2│ │2 3│ │3 1│ │3 2│ │3 3│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊──────────────────────────────────────────────────────┘

Primes between 1 and 100

In [107]:
(~R∊R∘.×R)/R←1↓⍳100

┌→─────────────────────────────────────────────────────────────────────┐
│2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97│
└~─────────────────────────────────────────────────────────────────────┘

## Composition (Currying)

We can curry a dyadic function down to a monadic function by fixing either left or right argument:

In [108]:
sum←{⍺+⍵}
add5←5∘sum

In [109]:
add5 7

12

In [110]:
add5 1 2 3 4 5 6 7

┌→───────────────┐
│6 7 8 9 10 11 12│
└~───────────────┘

## Shape and Rank

Pair consecutive elements in a vector

In [111]:
{⊂⍵}⌺(⍪2 2) ⊢ ⍳10

┌→───────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │3 4│ │5 6│ │7 8│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────┘

In [112]:
,⌿⍉5 2 ⍴ ⍳10

┌→───────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │3 4│ │5 6│ │7 8│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────┘

In [113]:
↓5 2 ⍴ ⍳10

┌→───────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 2│ │3 4│ │5 6│ │7 8│ │9 10│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
└∊───────────────────────────────┘

## Folds

The reduce operator / folds R-L. It's possible to define a fold operator that works L-R:

In [114]:
foldl←{↑⍺⍺⍨/(⌽⍵),⊂⍺}

In [115]:
0 + foldl 1 2 3 4 5 ⍝ Must give initial accumulator state

15

Note that this operator (and many others) exists in the standard Dyalog workspace [dfns](https://dfns.dyalog.com/s_foldl.htm), which can be imported as ⎕CY 'dfns'

## On ranges

In Python, for example, you can specify a [range](https://docs.python.org/3/library/functions.html#func-range) as `range(start, end)` which gives an iterator from start to end-1. In Dyalog there isn't a direct equivalent, but one can be made from the iota operator.

In [116]:
¯1+5 {⍺↓⍳⍵} 10   ⍝ Wasteful, as generating from ⎕IO

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [117]:
¯1+5 (⊣↓∘⍳) 10   ⍝ The abobve as a train

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [118]:
¯1+5 {⍺+⍳⍵-⍺} 10 ⍝ Better!

┌→────────┐
│5 6 7 8 9│
└~────────┘

In [119]:
¯1+5 (⊣+∘⍳-⍨) 10 ⍝ The above as a train

┌→────────┐
│5 6 7 8 9│
└~────────┘

## Stencils

Pick out 3x3 regions of a larger matrix. The stencil takes a function to the left and a shape to the right and returns a monad.

This from https://chat.stackexchange.com/rooms/52405/conversation/lesson-5-even-more-apl-operators--

In [120]:
⎕←letters ← 4 6⍴⎕A
{⊂⍵}⌺3 3 ⊢ letters     ⍝ Enclose the selected window

┌→─────┐
↓ABCDEF│
│GHIJKL│
│MNOPQR│
│STUVWX│
└──────┘

┌→────────────────────────────────────┐
↓ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ │ ↓ │ ↓ │ ↓ │ ↓ │ ↓ │ │
│ │ AB│ │ABC│ │BCD│ │CDE│ │DEF│ │EF │ │
│ │ GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ AB│ ↓ABC│ ↓BCD│ ↓CDE│ ↓DEF│ ↓EF │ │
│ │ GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL │ │
│ │ MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ GH│ ↓GHI│ ↓HIJ│ ↓IJK│ ↓JKL│ ↓KL │ │
│ │ MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR │ │
│ │ ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ ↓ MN│ ↓MNO│ ↓NOP│ ↓OPQ│ ↓PQR│ ↓QR │ │
│ │ ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX │ │
│ │ │ │ │ │ │ │ │ │ │ │ │ │
│ └───┘ └───┘ └───┘ └───┘ └───┘ └───┘ │
└∊────────────────────────────────────┘

The padding is defined by ⍺ in the operand function:

In [121]:
({⊂⍺}⌺3 3) ⊢ letters

┌→───────────────────────────────────────────┐
↓ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │1 1│ │1 0│ │1 0│ │1 0│ │1 0│ │1 ¯1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │0 1│ │0 0│ │0 0│ │0 0│ │0 0│ │0 ¯1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→───┐ │
│ │0 1│ │0 0│ │0 0│ │0 0│ │0 0│ │0 ¯1│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~───┘ │
│ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→───┐ ┌→────┐ │
│ │¯1 1│ │¯1 0│ │¯1 0│ │¯1 0│ │¯1 0│ │¯1 ¯1│ │
│ └~───┘ └~───┘ └~───┘ └~───┘ └~───┘ └~────┘ │
└∊───────────────────────────────────────────┘

In [122]:
({⊂⍺↓⍵}⌺3 3) ⊢ letters

┌→──────────────────────────────────┐
↓ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓AB│ ↓ABC│ ↓BCD│ ↓CDE│ ↓DEF│ ↓EF│ │
│ │GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
│ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓AB│ ↓ABC│ ↓BCD│ ↓CDE│ ↓DEF│ ↓EF│ │
│ │GH│ │GHI│ │HIJ│ │IJK│ │JKL│ │KL│ │
│ │MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
│ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓GH│ ↓GHI│ ↓HIJ│ ↓IJK│ ↓JKL│ ↓KL│ │
│ │MN│ │MNO│ │NOP│ │OPQ│ │PQR│ │QR│ │
│ │ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
│ ┌→─┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→─┐ │
│ ↓MN│ ↓MNO│ ↓NOP│ ↓OPQ│ ↓PQR│ ↓QR│ │
│ │ST│ │STU│ │TUV│ │UVW│ │VWX│ │WX│ │
│ └──┘ └───┘ └───┘ └───┘ └───┘ └──┘ │
└∊──────────────────────────────────┘

## Key

In [123]:
{⍺,⍵}⌸'Mississippi'  ⍝ Show indexes of items, by item

┌→──────────┐
↓M 1 │
│i 2 5 8 11│
│s 3 4 6 7│
│p 9 10 │
└+──────────┘

In [124]:
{⍺,≢⍵}⌸'Mississippi' ⍝ Frequencies

┌→──┐
↓M 1│
│i 4│
│s 4│
│p 2│
└+──┘

...or as tacit

In [125]:
(,∘≢⌸)'Mississippi'

┌→──┐
↓M 1│
│i 4│
│s 4│
│p 2│
└+──┘

Pick the most frequent item, returning all if several

In [126]:
{(⊣/m)/⍨frq⍷⍨frq⌷⍨⊃⍒frq←⊢/m←(,∘≢⌸)⍵} 'Mississippi' ⍝ Most frequent

┌→─┐
│is│
└──┘

## Sliding Tiles example

With ample help from @ngn on APL Orchard.

In [127]:
]dinput
moves←{
     d←(0 1)(1 0)(0 ¯1)(¯1 0)      ⍝ Move offsets
     m←⍵
     {⌽@i ⍵⊢m}¨(,⍳⍴m)∩d+⊂i←⊃⍸0=m
 }

In [130]:
state←¯1+4 4⍴?⍨16
moves state

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ ↓ 3 0 2 8│ ↓ 3 2 8 0│ ↓ 3 2 5 8│ │
│ │11 15 5 7│ │11 15 5 7│ │11 15 0 7│ │
│ │13 10 9 12│ │13 10 9 12│ │13 10 9 12│ │
│ │ 1 4 14 6│ │ 1 4 14 6│ │ 1 4 14 6│ │
│ └~──────────┘ └~──────────┘ └~──────────┘ │
└∊──────────────────────────────────────────┘

A lot to decode here. The `i` expression gives the row-col of the 0 element:

In [131]:
state ⋄ ⊃⍸0=state

┌→──────────┐
↓ 3 2 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘
┌→──┐
│1 3│
└~──┘

Adding the offsets gives all potential moves:

In [132]:
d←(0 1)(1 0)(0 ¯1)(¯1 0)
d+⊂i←⊃⍸0=state                ⍝ disclose-enclose as we want the enclosed version later

┌→────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 4│ │2 3│ │1 2│ │0 3│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊────────────────────────┘

Given the rank of m we can generate all valid coordinate pairs

In [133]:
,⍳⍴state

┌→────────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 1│ │1 2│ │1 3│ │1 4│ │2 1│ │2 2│ │2 3│ │2 4│ │3 1│ │3 2│ │3 3│ │3 4│ │4 1│ │4 2│ │4 3│ │4 4│ │
│ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ └~──┘ │
└∊────────────────────────────────────────────────────────────────────────────────────────────────┘

and then discard the moves that falls out of range as an intersection:

In [134]:
(,⍳⍴state)∩d+⊂i←⊃⍸0=state

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │1 2│ │1 4│ │2 3│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

We now map `{⌽@i ⍵⊢m}` over the list of valid moves. How does that work?

In [135]:
{⌽@i ⍵⊢state}¨(,⍳⍴state)∩d+⊂i←⊃⍸0=state

┌→──────────────────────────────────────────┐
│ ┌→──────────┐ ┌→──────────┐ ┌→──────────┐ │
│ ↓ 3 0 2 8│ ↓ 3 2 8 0│ ↓ 3 2 5 8│ │
│ │11 15 5 7│ │11 15 5 7│ │11 15 0 7│ │
│ │13 10 9 12│ │13 10 9 12│ │13 10 9 12│ │
│ │ 1 4 14 6│ │ 1 4 14 6│ │ 1 4 14 6│ │
│ └~──────────┘ └~──────────┘ └~──────────┘ │
└∊──────────────────────────────────────────┘

The dyadic `@` glyph in this case applies the reverse function `⌽` on the set of indexes given to its right argument. For example, if we want to switch the elements at (1 1) and (1 2) in m, we could say:

In [137]:
state ⋄ (⌽@(1 1) (1 2)) state

┌→──────────┐
↓ 3 2 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘
┌→──────────┐
↓ 2 3 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘

Using the right-tack `⊢` we can drop the brackets.

In [138]:
⌽@(1 1) (1 2) ⊢ state

┌→──────────┐
↓ 2 3 0 8│
│11 15 5 7│
│13 10 9 12│
│ 1 4 14 6│
└~──────────┘

## A heap implemented as a Leftist Tree

A heap is a data structure that can be implemented as a Leftist Tree, which lends itself naturally to a recursive implementation.

https://en.wikipedia.org/wiki/Leftist_tree
http://typeocaml.com/2015/03/12/heap-leftist-tree/

Here is a partial port of the OCaml version.

In [9]:
]dinput
Pop←{ ⍝ Pop off smallest element from a leftist tree ⍵
    0=≢⍵:⍬
    (value left right)←1↓⍵
    (left Merge right) value
}

In [10]:
]dinput
Push←{ ⍝ Insert item ⍵ into leftist tree ⍺, returning the resulting tree
    ⍺←⍬              ⍝ default to init
    1 ⍵ ⍬ ⍬ Merge ⍺ 
}

In [11]:
]dinput
Merge←{ ⍝ Merge leftist trees ⍺ and ⍵
    0=≢⍺:⍵ ⋄ 0=≢⍵:⍺                                 ⍝ If either is a leaf, return the other
    (key left right)←1↓⍺
    key>(⎕IO+1)⊃⍵:⍵∇⍺                               ⍝ Flip to ensure smallest is root of merged
    merged←right∇⍵                                  ⍝ Merge rightwards
    leftRank←⊃left ⋄ mergedRank←⊃merged
    leftRank≥⊃merged:(1+mergedRank) key left merged ⍝ Right is shorter
    (1+leftRank) key merged left                    ⍝ Left is shorter; make it the new right
}

In [12]:
⍝ Example given in http://typeocaml.com/2015/03/12/heap-leftist-tree/
h←Push 2
h Push←10
h Push←9

s←Push 3
s Push←6

h Merge s

┌─┬─┬─────────────────────┬────────┐
│2│2│┌─┬─┬───────┬───────┐│┌─┬──┬┬┐│
│ │ ││2│3│┌─┬─┬┬┐│┌─┬─┬┬┐│││1│10││││
│ │ ││ │ ││1│6│││││1│9│││││└─┴──┴┴┘│
│ │ ││ │ │└─┴─┴┴┘│└─┴─┴┴┘││ │
│ │ │└─┴─┴───────┴───────┘│ │
└─┴─┴─────────────────────┴────────┘

More compact, and a more natural fit for APL. Note that the Leftist Tree has slightly different performance characteristics to the standard heap. 

## Strings

Lower/upper-case a string

In [34]:
(819⌶) 'Hello All People in The World'

┌→────────────────────────────┐
│hello all people in the world│
└─────────────────────────────┘

In [35]:
1(819⌶) 'Hello All People in The World'

┌→────────────────────────────┐
│HELLO ALL PEOPLE IN THE WORLD│
└─────────────────────────────┘

We can also use regular expressions, of course:

In [36]:
{⊃('.+' ⎕S '\u0')⍵} 'Hello All People in The World'

┌→────────────────────────────┐
│HELLO ALL PEOPLE IN THE WORLD│
└─────────────────────────────┘

As we saw earlier, we can split a string on a regex pattern:

In [37]:
RegSplit←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵}

In [38]:
'/+' RegSplit 'here/////be/dragons/'

┌→──────────────────────────┐
│ ┌→───┐ ┌→─┐ ┌→──────┐ ┌⊖┐ │
│ │here│ │be│ │dragons│ │ │ │
│ └────┘ └──┘ └───────┘ └─┘ │
└∊──────────────────────────┘

...or simply on a separator char

In [39]:
' ' (≠⊆⊢) 'hello world out there'

┌→──────────────────────────────┐
│ ┌→────┐ ┌→────┐ ┌→──┐ ┌→────┐ │
│ │hello│ │world│ │out│ │there│ │
│ └─────┘ └─────┘ └───┘ └─────┘ │
└∊──────────────────────────────┘